In [1]:
from hcve_lib.utils import notebook_init

notebook_init()

import json
from typing import Dict

import yaml
from mlflow import set_tracking_uri, get_run, search_runs

chosen_runs = {
    'lco':
        {'coxnet': '582f3b16a31c42c08873a29634153869'}
}

from deps.hyperparameters import HYPERPARAMETERS_FILE

set_tracking_uri('http://localhost:5000')

def get_all_hyperparameters(run_id: str) -> Dict[str, Dict]:
    root_run = get_run(run_id)
    children_runs = search_runs(
        root_run.info.experiment_id,
        f'tags.mlflow.parentRunId = "{root_run.info.run_id}"',
        output_format='pandas')
    return {
        rows['tags.mlflow.runName']: json.loads(rows['params.hyperparameters1'].replace("\'", "\""))
        for _, rows in children_runs.iterrows()
    }


hyperparameters = {
    experiment_name: {
        method_name: get_all_hyperparameters(run_id)
        for method_name, run_id in methods.items()
    }
    for experiment_name, methods in chosen_runs.items()
}

print(yaml.dump(hyperparameters))

with open(HYPERPARAMETERS_FILE, 'w') as f:
    json.dump(hyperparameters, f)


KeyError: 'params.hyperparameters'